In [6]:
import torch
import torchvision
from torchvision import datasets, transforms
from torchvision.datasets import CIFAR10
from torchvision.models import resnet18, ResNet18_Weights
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
model = resnet18(weights=ResNet18_Weights.DEFAULT)

# Modify the final fully connected layer for CIFAR-10 (10 classes)
model.fc = torch.nn.Linear(model.fc.in_features, 10)

# Move model to device
model.to(device)

# CIFAR-10 Dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match ResNet's input size
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # Standard normalization for ResNet
])

trainset = CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)






Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Loss: 0.48088605296047754
Accuracy on CIFAR-10: 92.16%


In [12]:
%%time
for epoch in range(2): 
    model.train()
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}")

Epoch 1, Loss: 0.1894123461049841
CPU times: user 3h 10min 30s, sys: 4h 3min 15s, total: 7h 13min 46s
Wall time: 56min 53s


In [ ]:
%%time
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on CIFAR-10: {100 * correct / total:.2f}%")